In [1]:
function newton(f,g,h,ϵg,x0,M)
    x1=x0
    gx=g(x1...)
    hx=h(x1...)
    d=-inv(hx)*gx
    ng=norm(d)
    i=1
    println("$x1\t$gx")
    while (ng>ϵg &&i<M)
        x2=x1+d
        i+=1
        x1=x2
        gx=g(x1...)
        d=-inv(hx)*gx
        ng=norm(d)
        println("$x1\t$gx")
    end
    return x1,f(x1...)
end

newton (generic function with 1 method)

In [ ]:
newton((x1,x2,x3,x4)->x1-0.6*x2+4*x3+0.25*x4-(log(x1)+log(x2)+log(x3)+log(x4))-log(5-(log(x1)+log(x2)+log(x3)+log(x4)))
       (x1,x2,x3,x4)->[1/(x1*(-log(x1)-log(x2)-log(x3)-log(x4)+5*log(10)))+1-1/x1 1/(x2*(-log(x1)-log(x2)-log(x3)-log(x4)+5*log(10)))-0.6-1/x2;
                       1/(x3*(-log(x1)-log(x2)-log(x3)-log(x4)+5*log(10)))+4-1/x3 1/(x4*(-log(x1)-log(x2)-log(x3)-log(x4)+5*log(10)))+0.25-1/x4],
        (x1,x2,x3,x4)->)

In [43]:
### Rosenbrock  f(x,y)=(1-x)^2+100*(y-x^2)^2
newton((x,y)->(1-x)^2+100*(y-x^2)^2,(x,y)->[2*(x-1)-400*x*(y-x^2),200*(y-x^2)],(x,y)->[2+800*x^2-400*(y-x^2) -400*x;-400*x 200],0.001,[0,2],10)

[0, 2]	[-2, 400]
[-0.00250627, 0.0]	[-2.00502, -0.00125627]
[-0.00501882, 6.28137e-6]	[-2.01008, -0.00378144]
[-0.00753771, 2.51886e-5]	[-2.01517, -0.00632571]
[-0.010063, 5.68171e-5]	[-2.0203, -0.00888933]
[-0.0125947, 0.000101264]	[-2.02548, -0.0114725]
[-0.0151329, 0.000158626]	[-2.03069, -0.0140756]
[-0.0176776, 0.000229004]	[-2.03595, -0.0166988]
[-0.0202289, 0.000312498]	[-2.04124, -0.0193423]
[-0.0227869, 0.00040921]	[-2.04658, -0.0220064]


([-0.0227869, 0.00040921], 1.0460942025363027)

In [1]:
function inexact_method(f,g,ϕ0,dϕ0,xk,dk;τ=0.5,ϵ=0.5,ζ=2)
    α=1
    dα=dk'*g((xk.+α.*dk)...)##dk表示梯度/对α求导
    while (dα<0)
        α=ζ*α
        dα=dk'*g((xk.+α.*dk)...)
    end
    ϕα=f((xk.+α.*dk)...)
    while (ϕα>ϕ0+ϵ*α*dϕ0)
        α=τ*α
        ϕα=f((xk.+α.*dk)...)
    end
    return α
end

inexact_method (generic function with 1 method)

In [45]:
function  Levenberg_Marquardt(f,g,h,ϵg,x0,λ,M)
    x1=x0
    gx=g(x1...)
    hx=h(x1...)
    f1=f(x1...)
    I=eye(2,2)
    H=hx+λ.*I
    d=-inv(H)*gx
    ng=norm(d)
    i=1
    println("step=",i,"\tx=",x1, "\tg=",gx, "\t\tf=",f1)
    while (ng>ϵg &&i<M)
        ####判断黑塞矩阵是否正定
        if (eigvals(H)[1]*eigvals(H)[2]>0)
            α=inexact_method(f,g,f1,-d'*d,x1,d)
            x2=x1+α.*d
            x1=x2
            i+=1
            f1=f(x1...)
            gx=g(x1...)
            hx=h(x1...)    
            H=hx+λ.*I
            d=-inv(H)*gx
            ng=norm(d)
            println("step=",i,"\tα=",α,"\tx=",x1, "\tg=",gx, "\t\tf=",f1)
        else
           ###若是不正定取原搜索方向的正交方向为新的搜索方向
           d=[d[2],d[1]]
           α=inexact_method(f,g,f1,-d'*d,x1,d)
           x2=x1+α.*d
           x1=x2
           i+=1
           gx=g(x1...)
           hx=h(x1...) 
           f1=f(x1...)
           H=hx+λ.*I
           d=-inv(H)*gx
           ng=norm(d)
           println("step=",i,"\tα=",α,"\tx=",x1, "\tg=",gx, "\t\tf=",f1)
            if (eigvals(H)[1]*eigvals(H)[2]<0)
                #####若还是不正定，取搜索方向的反方向
                d=-d
                α=inexact_method(f,g,f1,-d'*d,x1,d)
                x2=x1+α.*d
                x1=x2
                i+=1
                gx=g(x1...)
                hx=h(x1...) 
                f1=f(x1...)
                H=hx+λ.*I
                d=-inv(H)*gx
                ng=norm(d)     
                println("step=",i,"\tα=",α,"\tx=",x1,"\tg=",gx, "\t\tf=",f1)
            end
        end
    end
    return x1,f(x1...)
end    

Levenberg_Marquardt (generic function with 1 method)

In [46]:
### Rosenbrock  f(x,y)=(1-x)^2+100*(y-x^2)^2
##λ=0  =>牛顿法
Levenberg_Marquardt((x,y)->(1-x)^2+100*(y-x^2)^2,(x,y)->[400*x^3-400*x*y+2*x-1,200*(y-x^2)],(x,y)->[2+800*x^2-400*(y-x^2)-400*x -400*x;-400*x 200],
    0.001,[-2,2],0,128)

step=1	x=[-2, 2]	g=[-1605, -400]		f=409
step=2	α=1.0	x=[-1.99688, 3.98752]	g=[-5.00154, -0.00194825]		f=8.981283158985544
step=3	α=32.0	x=[-1.79732, 3.19083]	g=[-33.0087, -7.90459]		f=7.98118903916589
step=4	α=2	x=[-1.78484, 3.22504]	g=[23.5362, 7.87348]		f=7.910335714529468
step=5	α=2	x=[-1.77179, 3.09971]	g=[-32.5646, -7.90755]		f=7.839150968430205
step=6	α=2	x=[-1.75928, 3.13446]	g=[23.1946, 7.87626]		f=7.768735896205096
step=7	α=2	x=[-1.74619, 3.00961]	g=[-32.119, -7.91057]		f=7.697979231452115
step=8	α=2	x=[-1.73364, 3.04491]	g=[22.8518, 7.87911]		f=7.628001701091245
step=9	α=2	x=[-1.7205, 2.92054]	g=[-31.6719, -7.91367]		f=7.557671999931443
step=10	α=2	x=[-1.70792, 2.95639]	g=[22.5078, 7.88201]		f=7.488131233796336
step=11	α=2	x=[-1.69472, 2.83251]	g=[-31.2231, -7.91683]		f=7.418227286245081
step=12	α=2	x=[-1.68211, 2.8689]	g=[22.1625, 7.88498]		f=7.349122433688706
step=13	α=2	x=[-1.66886, 2.7455]	g=[-30.7727, -7.92006]		f=7.279642929495505
step=14	α=2	x=[-1.6562, 2.78245]	g=[21.

([0.156533, -0.00379401], 0.7915063361701797)

In [47]:
### Rosenbrock  f(x,y)=(1-x)^2+100*(y-x^2)^2
##λ->无穷大   =>梯度下降法
Levenberg_Marquardt((x,y)->(1-x)^2+100*(y-x^2)^2,(x,y)->[400*x^3-400*x*y+2*x-1,200*(y-x^2)],(x,y)->[2+800*x^2-400*(y-x^2)-400*x -400*x;-400*x 200],
    0.001,[-2,2],10^6,128)

step=1	x=[-2, 2]	g=[-1605, -400]		f=409
step=2	α=512	x=[-1.18233, 2.20411]	g=[377.914, 161.24]		f=69.75863872348746


([-1.18233, 2.20411], 69.75863872348746)